Download colab_utils and import



<a href="https://colab.research.google.com/github/beangoben/rdkit_colab/blob/master/Template.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>


In [ ]:
!wget https://raw.githubusercontent.com/beangoben/workshop_template/master/colab_utils.py -O colab_utils.py
!rm -rf sample_data
import colab_utils

Clone github repo, install stuff

In [2]:
github_repo = 'https://github.com/beangoben/rdkit_colab'
colab_utils.clone_repo(github_repo)
colab_utils.conda_install_from_yaml()
colab_utils.pip_install_from_yaml()

CMD: git clone https://github.com/beangoben/rdkit_colab
Conda installing ['umap-learn', 'rdkit']
 from channels ['rdkit', 'conda-forge']
['rdkit', 'conda-forge']
['conda install -q -y  -c rdkit -c conda-forge umap-learn', 'conda install -q -y  -c rdkit -c conda-forge rdkit']
pip installing ['e3fp', 'selfies']
CMD: wget -c https://repo.continuum.io/miniconda/Miniconda3-latest-Linux-x86_64.sh
CMD: chmod +x Miniconda3-latest-Linux-x86_64.sh
CMD: bash ./Miniconda3-latest-Linux-x86_64.sh -b -f -p /usr/local
PREFIX=/usr/local
Unpacking payload ...
Solving environment: ...working... done
## Package Plan ##
environment location: /usr/local
added / updated specs:
- _libgcc_mutex==0.1=main
- asn1crypto==1.3.0=py37_0
- ca-certificates==2020.1.1=0
- certifi==2019.11.28=py37_0
- cffi==1.14.0=py37h2e261b9_0
- chardet==3.0.4=py37_1003
- conda-package-handling==1.6.0=py37h7b6447c_0
- conda==4.8.2=py37_0
- cryptography==2.8=py37h1ba5d50_0
- idna==2.8=py37_0
- ld_impl_linux-64==2.33.1=h53a641e_7
- libed

# Import modules


In [5]:
from collections import OrderedDict
import tqdm.auto as tqdm

import numpy as np
import scipy as sp
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns


import tensorflow as tf

colab_utils.print_module_versions()
colab_utils.matplotlib_settings()

rdkit  : 2020.03.1
tf     : 2.2.0-rc4
